In [5]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv('athletes.csv')

In [4]:
df.dropna(inplace=True)

In [6]:
def create_enc(df, columns):
    for col in columns:
        df[col+'_enc'] = df[col]
        encoder = LabelEncoder()
        encoder.fit(df[col])
        df[col+'_enc'] = encoder.transform(df[enc])
    return df

In [9]:
create_enc(df, ['sport', 'nationality'])

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze,sport_enc,nationality_enc
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0,athletics,ESP
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0,fencing,KOR
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1,athletics,CAN
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0,taekwondo,MDA
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0,cycling,NZL
5,173071782,Aaron Royle,AUS,male,1/26/90,1.80,67.0,triathlon,0,0,0,triathlon,AUS
6,266237702,Aaron Russell,USA,male,6/4/93,2.05,98.0,volleyball,0,0,1,volleyball,USA
7,382571888,Aaron Younger,AUS,male,9/25/91,1.93,100.0,aquatics,0,0,0,aquatics,AUS
8,87689776,Aauri Lorena Bokesa,ESP,female,12/14/88,1.80,62.0,athletics,0,0,0,athletics,ESP
9,997877719,Ababel Yeshaneh,ETH,female,7/22/91,1.65,54.0,athletics,0,0,0,athletics,ETH


In [10]:
df.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze,sport_enc,nationality_enc
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0,athletics,ESP
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0,fencing,KOR
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1,athletics,CAN
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0,taekwondo,MDA
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0,cycling,NZL
